In [2]:
pip install langchain

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install langchain_community

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install langchain_openai

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install html2text

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install bs4

  Using cached bs4-0.0.2-py2.py3-none-any.whl.metadata (411 bytes)
Using cached bs4-0.0.2-py2.py3-none-any.whl (1.2 kB)
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install chromadb

Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import AsyncHtmlLoader
from langchain_community.document_transformers import Html2TextTransformer
from langchain.chains import RetrievalQA
from langchain_openai import OpenAI
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

# load url and transform the text
loader = AsyncHtmlLoader("https://www.nasa.gov/news/all-news/")
data = loader.load()
html2text = Html2TextTransformer()
docs_transformed = html2text.transform_documents(data)
new_docs = docs_transformed[0].page_content[0:10000]


Fetching pages: 100%|##########| 1/1 [00:00<00:00,  6.00it/s]


In [3]:
# split the text using RecursiveCharacterTextSplitter
rc_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap= 5,
    separators=['\n']
)
docs = rc_splitter.split_text(new_docs)

In [4]:
# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# set the persist directory
persist_directory='D:\ISU Courses\IT 496\persist_directory'

# store embedding into vectorstore using Chroma
vectordb = Chroma(
    persist_directory = persist_directory,
    embedding_function = embedding_function
)

<>:5: SyntaxWarning: invalid escape sequence '\I'
<>:5: SyntaxWarning: invalid escape sequence '\I'
C:\Users\sabbi\AppData\Local\Temp\ipykernel_25856\87099030.py:5: SyntaxWarning: invalid escape sequence '\I'
  persist_directory='D:\ISU Courses\IT 496\persist_directory'
c:\Users\sabbi\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\sabbi\AppData\Local\Programs\Python\Python312\Lib\site-packages\onnxruntime\capi\onnxruntime_validation.py:26: UserWarning: Unsupported Windows version (11). ONNX Runtime supports Windows 10 and above, only.
  warnings.warn(


In [5]:
# persist to memory
vectordb.persist()
docstorage = Chroma.from_texts(docs, embedding_function)


In [7]:
# Invoke prompt using RetrievalQA
qa = RetrievalQA.from_chain_type(OpenAI(temperature=0.7,base_url="http://localhost:1234/v1", api_key="not-needed"),chain_type='stuff', retriever=docstorage.as_retriever())

# show response to screen
def process_messages(prompt):
    response = qa({"query": prompt})

    return response['result']

if __name__ == "__main__":
    while True:
        human_input = input("Human: ")

        if human_input.lower() in ["quit", "exit", "bye"]:
            break

        print("Thinking...")
        response = process_messages(human_input)
        print("Chatbot: ", response)

Thinking...
Chatbot:  
